# Seyed Ali Mirferdos
# 99201465

# 1. Importing required modules

In [1]:
!pip install mmh3

     |████████████████████████████████| 51kB 6.5MB/s 


In [28]:
import pandas as pd
import mmh3
import statistics
import math
import random

# 2. Reading the data

In [5]:
!unzip archive.zip

Archive:  archive.zip
  inflating: data.csv                


In [6]:
df = pd.read_csv('data.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df.head()

,host,time,method,url,response,bytes
0,***.novo.dk,805465029,GET,/ksc.html,200,7067
1,***.novo.dk,805465031,GET,/images/ksclogo-medium.gif,200,5866
2,***.novo.dk,805465051,GET,/images/MOSAIC-logosmall.gif,200,363
3,***.novo.dk,805465053,GET,/images/USA-logosmall.gif,200,234
4,***.novo.dk,805465054,GET,/images/NASA-logosmall.gif,200,786


In [8]:
hosts = df['host']

In [27]:
len(hosts.unique())

135268

# Part A: Flajolet-Martin

The base of following code was taken from [here](https://github.com/javiajinkal/Flajolet-Martin/blob/master/Flajolet-Martin.py)

In [9]:
def trailing_zeros(n):
	s = str(n)
	return len(s) - len(s.rstrip('0'))

In [24]:
def flajolet_martin(data, L):
  print(f'L is: {L}')
  groups = [i for i in range(2, L + 1) if L % i == 0]
  result_tail = [0] * L

  for _, val in data.items():
    for seed in range(L):
      result = format(abs(mmh3.hash(val, seed)), '032b')
      result_tail[seed] = max(result_tail[seed], 
                              trailing_zeros(result))

  for i in groups:
    groups_val = []
    size = L // i
    for idx in range(i):
      chunk = [2**(result_tail[j]) for j in range(size * idx, size * (idx + 1))]
      groups_val.append(statistics.mean(chunk))
    
    print(f'The median of avg for groups of size {size}:',
          math.ceil(statistics.median(groups_val)))

In [25]:
for i in range(8, 36, 8):
  flajolet_martin(hosts, i)

L is: 8
The median of avg for groups of size 4: 262144
The median of avg for groups of size 2: 294912
The median of avg for groups of size 1: 131072
L is: 16
The median of avg for groups of size 8: 372736
The median of avg for groups of size 4: 352256
The median of avg for groups of size 2: 294912
The median of avg for groups of size 1: 131072
L is: 24
The median of avg for groups of size 12: 282624
The median of avg for groups of size 8: 262144
The median of avg for groups of size 6: 305835
The median of avg for groups of size 4: 262144
The median of avg for groups of size 3: 196608
The median of avg for groups of size 2: 212992
The median of avg for groups of size 1: 131072
L is: 32
The median of avg for groups of size 16: 284672
The median of avg for groups of size 8: 276480
The median of avg for groups of size 4: 262144
The median of avg for groups of size 2: 262144
The median of avg for groups of size 1: 131072


# Part B: Alon-Matias-Szegedy

In [77]:
stream_size = hosts.shape[0]
Ls = [3, 10, 30, 50, 60, 100, 200]
sample_times = {}
for L in Ls:
  sample_times[L] = {'times': random.sample(range(stream_size), L),
                     'vals': [''] * L,
                     'counts': [0] * L}

for t, val in hosts.items():
  for L, obj in sample_times.items():
    for i in range(len(obj['times'])):
      if obj['times'][i] <= t:
        if obj['vals'][i] == '':
          obj['vals'][i] = val
          obj['counts'][i] += 1
        elif obj['vals'][i] == val:
          obj['counts'][i] += 1


for k, L in sample_times.items():
  m2 = stream_size * statistics.mean([2 * c - 1 for c in L['counts']])
  print(f'2nd moment est. with L = {k}: {m2}')

2nd moment est. with L = 3: 145312489
2nd moment est. with L = 10: 1019559871.8000001
2nd moment est. with L = 30: 2729502344.4
2nd moment est. with L = 50: 2228441157.84
2nd moment est. with L = 60: 4458424407.400001
2nd moment est. with L = 100: 1120566879.46
2nd moment est. with L = 200: 2732883083.94
